In [1]:
import time
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## Read input data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
## Hyper parameters
EPOCHS = 70000
LEARNING_RATE = 0.001
N_BATCHES = 1 # Stochastic


In [5]:
## Construct model
class LearningModel:
    def __init__(self, sess):
        self.sess = sess
        self.build_model()
        
    def build_model(self):
        self.X = tf.placeholder(tf.float32, [None, 784])
        self.Y = tf.placeholder(tf.float32, [None, 10])

        W = tf.Variable(tf.zeros([784, 10]))
        b = tf.Variable(tf.zeros([10]))

        self.model = tf.add(tf.matmul(self.X, W), b)

        # define cost and optimizer
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.model, labels=self.Y))
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE).minimize(self.cost)

        # define accuracy
        prediction = tf.equal(tf.argmax(self.model, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
        
    def do_predict(self, x_test):
        return self.sess.run(self.model, feed_dict={self.X:x_test})
    
    def get_accuracy(self, x_test, y_test):
        return self.sess.run(self.accuracy, feed_dict={self.X:x_test, self.Y:y_test})
    
    def do_train(self, x_test, y_test):
        return self.sess.run([self.optimizer, self.cost], feed_dict={self.X:x_test, self.Y:y_test})



In [6]:
with tf.Session() as sess:
    mymodel = LearningModel(sess)
    sess.run(tf.global_variables_initializer())
    
    ## Training
    input_size = mnist.train.images.shape[0]
    t_start = time.time()
    for e in range(EPOCHS):
        total_cost = 0

        input_idx = np.random.choice(input_size, N_BATCHES)
        batch_xs = mnist.train.images[input_idx]
        batch_ys = mnist.train.labels[input_idx]

        _, cost_val = mymodel.do_train(batch_xs, batch_ys)
        total_cost += cost_val

        if e % 5000 == 0:
            print(e, 'Avgerage cost = ', '{:.3f}'.format(total_cost))
    print("\nTraining time: %f secs" % (time.time() - t_start))
    
    ## Test
    predict = mymodel.do_predict(mnist.test.images)
    accuracy = mymodel.get_accuracy(mnist.test.images, mnist.test.labels)
    print('accuracy: ', accuracy)

0 Avgerage cost =  2.303
5000 Avgerage cost =  0.688
10000 Avgerage cost =  0.291
15000 Avgerage cost =  0.030
20000 Avgerage cost =  0.093
25000 Avgerage cost =  0.053
30000 Avgerage cost =  0.730
35000 Avgerage cost =  0.100
40000 Avgerage cost =  0.061
45000 Avgerage cost =  0.128
50000 Avgerage cost =  0.154
55000 Avgerage cost =  0.022
60000 Avgerage cost =  0.186
65000 Avgerage cost =  0.070

Training time: 74.391928 secs
accuracy:  0.9058
